<a href="https://colab.research.google.com/github/SrushtiGunjal/whatsapp_bots/blob/main/whatsapp_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === WhatsApp CSV-to-Graph Bot (Only Graph Mode) ===
# Dependencies (already installed in your case)
from flask import Flask, request, send_from_directory
from twilio.twiml.messaging_response import MessagingResponse
from pyngrok import ngrok
import pandas as pd
import matplotlib.pyplot as plt
import os

# === Flask App Initialization ===
app = Flask(__name__)

# === Ngrok Auth Token Setup ===
ngrok.set_auth_token("2yiU4c698ARDNROumEtQVBfpQpu_3WwV6ZpQkZxcZNSuts6M3")
public_url = ngrok.connect(5000).public_url
print("ngrok tunnel:", public_url)

# === Load CSV ===
try:
    df = pd.read_csv('/content/fact_dpos_1_day.csv')
    df['date_id'] = pd.to_datetime(df['date_id'])
    column_names = df.columns.tolist()
except FileNotFoundError:
    print("[ERROR] CSV file not found.")
    df = pd.DataFrame()
    column_names = []

# === Keywords for Graph Types ===
GRAPH_KEYWORDS = ['plot', 'graph', 'line', 'bar', 'pie', 'chart', 'distribution']

def generate_graph_response(prompt):
    import matplotlib
    matplotlib.use('Agg')
    plt.style.use('fivethirtyeight')
    prompt = prompt.lower()
    fig, ax = plt.subplots(figsize=(14, 6))

    if df.empty:
        plt.close(fig)
        return None, "⚠️ CSV data not loaded."

    try:
        if "line" in prompt and "site" in prompt and 'site' in df.columns and 'mw' in df.columns and 'date_id' in df.columns:
            sites_to_plot = df['site'].unique()[:5]
            for site in sites_to_plot:
                timeseries = df[df['site'] == site].groupby('date_id')['mw'].sum()
                ax.plot(timeseries.index, timeseries.values, label=site)
            ax.set_title("MW Over Time (Sample Sites)")
            ax.set_xlabel("Date")
            ax.set_ylabel("MW")
            ax.legend()
            plt.xticks(rotation=45, ha='right')

        elif "bar" in prompt and "site" in prompt and 'site' in df.columns and 'mw' in df.columns:
            grouped = df.groupby('site')['mw'].sum().sort_values(ascending=False).head(20)
            ax.bar(grouped.index, grouped.values)
            ax.set_title("MW by Site")
            plt.xticks(rotation=45, ha='right')

        elif "pie" in prompt and "section" in prompt and 'section' in df.columns and 'mw' in df.columns:
            grouped = df.groupby('section')['mw'].sum()
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.pie(grouped.values, labels=grouped.index, autopct='%1.1f%%')
            ax.set_title("MW Distribution by Section")

        else:
            plt.close(fig)
            return None, "⚠️ Try 'line graph for site', 'bar graph for site', or 'pie chart for section'."

        plt.tight_layout()
        image_dir = "static"
        os.makedirs(image_dir, exist_ok=True)
        image_path = os.path.join(image_dir, "graph.png")
        fig.savefig(image_path)
        plt.close(fig)
        return image_path, "📊 Graph generated successfully."

    except Exception as e:
        print("[ERROR]:", e)
        plt.close(fig)
        return None, f"❌ Error generating graph: {e}"

@app.route('/static/<filename>')
def serve_static(filename):
    return send_from_directory('static', filename)

@app.route("/bot", methods=["POST"])
def bot():
    try:
        msg = request.form.get('Body')
        sender = request.form.get('From')
        print(f"Message from {sender}: {msg}")

        response = MessagingResponse()

        if any(word in msg.lower() for word in GRAPH_KEYWORDS):
            img_path, reply_text = generate_graph_response(msg)
            message = response.message(reply_text)
            if img_path:
                message.media(f"{public_url}/static/graph.png")
            else:
                response.message(reply_text)
        else:
            response.message("Hi! Send a graph query like 'line graph for site' or 'pie chart for section'.")

        return str(response)

    except Exception as e:
        print("[ERROR in /bot]:", e)
        response = MessagingResponse()
        response.message("❌ Internal server error.")
        return str(response)

if __name__ == "__main__":
    os.makedirs('static', exist_ok=True)
    app.run(port=5000, debug=True)